In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal
import pywt #信号处理库
import os
from train_funcs import *

In [ ]:
#数据的读取
data_all_ = np.array(pd.read_csv('original_data.csv'))
print(data_all_.shape)

In [ ]:
#查看相应库的版本
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

In [ ]:
#通过加入噪声来扩充样本
data_all = np.zeros((data_all_.shape[0] * 2,data_all_.shape[1]))
for i in range(data_all_.shape[0]):
    for j in range(data_all_.shape[1] - 1):
        add_percent1 = 1 + np.random.uniform(low=-0.1, high=0.1)
        add_percent2 = 1 + np.random.uniform(low=-0.1, high=0.1)
        add_percent3 = 1 + np.random.uniform(low=-0.1, high=0.1)
        add_percent4 = 1 + np.random.uniform(low=-0.1, high=0.1)
        add_percent5 = 1 + np.random.uniform(low=-0.1, high=0.1)
        data_all[i,j] = data_all_[i,j]
        data_all[i,-1] = data_all_[i,-1]
        data_all[i+data_all_.shape[0],j] = data_all_[i,j] * add_percent1
        data_all[i+data_all_.shape[0],-1] = data_all_[i,-1]

print(data_all.shape)

In [ ]:
#进行数据的划分
data_len = data_all.shape[1] - 1
data_sample = data_all.shape[0]

import pandas as pd
np.random.shuffle(data_all)#数据打乱

df1 = pd.DataFrame(data_all[:,:-1])
lbls = data_all[:,-1]

import numpy as np

def one_hot(y):
    lbl = np.zeros(6) #分类个数
    lbl[y] = 1
    return lbl

target = []
for value in lbls:
    target.append(one_hot(int(value)))
target = np.array(target)
wave = np.expand_dims(np.array(df1), axis=-1)

print(wave.shape)
print(target.shape)

In [ ]:
#导入LightTCN网络
from LightTCN import *

model = LightTCN()
print(model.summary())#查看网络结构

#开始进行训练
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras import layers
from keras import regularizers
import os
import keras
import keras.backend as K

In [ ]:
#训练参数的设定
X = wave
y = target
EPOCHS = 1000
BATCH_SIZE = 8
VALIDATION_SPLIT = 0.3

adam = keras.optimizers.Adam()#优化器

model.compile(optimizer=adam,
              loss="categorical_crossentropy", metrics=["acc"])#模型编译

def lr_scheduler(epoch):
    '''
    函数作用：每隔25个epoch，学习率减小为原来的0.5
    输入：epoch轮次
    输出：改变后的学习率
    '''
    if epoch % 25 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.5)
        print("lr changed to {}".format(lr * 0.5))
    return K.get_value(model.optimizer.lr)

lrate = LearningRateScheduler(lr_scheduler)

# filepath="model/{epoch:02d}-{val_acc:.2f}.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
# mode='max')
# callbacks_list = [checkpoint]

In [ ]:
#模型的训练与保存
history = model.fit(X, y, epochs=EPOCHS,
                    batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
                    verbose=1, callbacks=[lrate])
save_path = 'model/model.h5'
model.save(save_path)

In [ ]:
#绘制acc和loss曲线
print(history.history.keys())
# summarize history for accuracy
# plt.figure(figsize=(10,5),dpi=300)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.savefig('acc.png')
plt.show()
# summarize history for loss
# plt.figure(figsize=(10,5),dpi=300)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.savefig('loss.png')
plt.show()